<a href="https://colab.research.google.com/github/mission-impozzible/learning/blob/main/network_analysis/2025_11_26_network_analysis_community_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Analysis

1. Introduzione
1. Cos'è una rete a invarianza di scala?
1. Cosa si intende per reti orientate e il concetto di spazializzazione
1. Cos'è una ricevuta bancaria?
1. Quanti sono gli ateco e le filiere del MMIIT
1. Spiegami l'algoritmo di Louvain
1. Spiegami l'algoritmo Spinglass
1. Quali alternative esistono?
1. Spiegami il concetto di modularità
1. Estraimi i principali insights da questo paper (Newman, 2006)

**References**  
Newman, M. E. (2006). Modularity and community structure in networks. Proceedings of the national academy of sciences, 103(23), 8577-8582.


# Community Detection

La **Community Detection** (o rilevazione di comunità) è un processo fondamentale nell'ambito della network analysis il cui scopo è **identificare raggruppamenti di nodi** all'interno di una rete complessa. Questi gruppi, noti come comunità o cluster, si distinguono perché i loro membri sono **più densamente interconnessi** tra loro rispetto ai nodi del resto della rete. L'obiettivo primario di questa analisi è svelare la **struttura a comunità intrinseca** o la modularità della rete. Tali sottostrutture riflettono spesso moduli significativi o unità funzionali nel mondo reale, come le cerchie sociali in un social network, i gruppi di ricerca che collaborano in una rete di co-autori, o i moduli di proteine interagenti in una rete biologica.

Le comunità sono definite in base al principio di connettività differenziale: i nodi appartenenti alla stessa comunità sono caratterizzati da **connessioni intra-comunità** molto più numerose e dense, che li legano fortemente l'uno all'altro. Al contrario, le **connessioni inter-comunità**, ovvero i legami tra nodi appartenenti a comunità diverse, sono relativamente **scarse e deboli**. Questa differenza nella densità di collegamento è il segnale che permette agli algoritmi di distinguere i confini tra i vari gruppi. La Community Detection è, in sintesi, uno strumento cruciale per la **riduzione della complessità** e per l'interpretazione dei dati nelle reti, offrendo una mappa più leggibile e significativa della loro organizzazione interna.


Poiché il numero e la dimensione delle comunità sono solitamente sconosciuti *a priori*, la loro identificazione è un problema computazionalmente difficile che viene affrontato con una varietà di metodi. Tra gli approcci più diffusi vi è l'**ottimizzazione della Modularità**, che impiega algoritmi come il Metodo Louvain per massimizzare una misura ($M$) che valuta la qualità della partizione confrontando i legami interni alle comunità con quelli attesi in una rete casuale. Altri metodi includono le **Tecniche Spettrali**, che utilizzano gli autovettori di matrici associate al grafo (come la matrice Laplaciana) per la clusterizzazione, e gli **Algoritmi basati sulle Clique**, che identificano e analizzano le sovrapposizioni di sottografi altamente connessi.



In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from community import community_louvain # Necessita l'installazione: pip install python-louvain

# --- 1. Generazione del Dataset Simulato ---

# 10 settori aziendali di esempio
settori = [
    'Manifattura', 'Materie Prime', 'Trasporti & Logistica',
    'Distribuzione', 'Retail', 'Servizi IT',
    'Finanza', 'Agricoltura', 'Energia', 'Consulenza'
]

# Numero totale di transazioni simulate (fatture)
num_transazioni = 1000

# Creazione di transazioni casuali
np.random.seed(42)
data = {
    'Settore_Creditore': np.random.choice(settori, num_transazioni),
    'Settore_Debitore': np.random.choice(settori, num_transazioni),
    'Importo': np.random.randint(100, 50000, num_transazioni),
    'Connessioni': 1 # Ogni riga rappresenta 1 connessione
}

df_transazioni = pd.DataFrame(data)

# Rimuovi le transazioni di un settore verso sé stesso (opzionale, ma tipico in una filiera)
df_transazioni = df_transazioni[df_transazioni['Settore_Creditore'] != df_transazioni['Settore_Debitore']]

# Aggregazione: sommiamo gli importi e contiamo le connessioni per ogni coppia unica Creditore -> Debitore
df_network = df_transazioni.groupby(['Settore_Creditore', 'Settore_Debitore']).agg(
    Numero_Connessioni=('Connessioni', 'sum'),
    Somma_Importi=('Importo', 'sum')
).reset_index()

# Visualizzazione del DataFrame aggregato
print("--- DataFrame Aggregato (Transazioni per Coppia di Settori) ---")
print(df_network.head())
print(f"\nCoppie di relazioni uniche: {len(df_network)}")

--- DataFrame Aggregato (Transazioni per Coppia di Settori) ---
  Settore_Creditore Settore_Debitore  Numero_Connessioni  Somma_Importi
0       Agricoltura       Consulenza                  14         282835
1       Agricoltura    Distribuzione                   4         128662
2       Agricoltura          Energia                  12         240489
3       Agricoltura          Finanza                   9         261515
4       Agricoltura      Manifattura                  10         272158

Coppie di relazioni uniche: 90


In [ ]:
# --- 2. Costruzione del Grafo (NetworkX) ---

# Creazione del grafo diretto (DiGraph) poiché la transazione ha una direzione
G = nx.DiGraph()

# Aggiunta degli archi (edges) dal DataFrame
# Usiamo 'Numero_Connessioni' come peso (weight) dell'arco
for index, row in df_network.iterrows():
    G.add_edge(
        row['Settore_Creditore'], # Nodo Sorgente
        row['Settore_Debitore'], # Nodo Destinazione
        weight=row['Numero_Connessioni'], # Peso: Numero di fatture
        amount=row['Somma_Importi']      # Attributo aggiuntivo: Somma degli importi
    )

print("\n--- Informazioni di Base sul Grafo ---")
print(f"Numero di Nodi (Settori): {G.number_of_nodes()}")
print(f"Numero di Archi (Relazioni uniche): {G.number_of_edges()}")

# Esempio di calcolo di una metrica: Centralità di Grado (Degree Centrality)
# Questa misura identifica i settori con più partner commerciali (in entrata e in uscita)
degree_centrality = nx.degree_centrality(G)
# Ordina i settori per centralità
top_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)

print("\n--- Centralità di Grado (Top 3 Settori) ---")
for sector, centrality in top_centrality[:3]:
    print(f"Settore: {sector}, Centralità: {centrality:.4f}")


--- Informazioni di Base sul Grafo ---
Numero di Nodi (Settori): 10
Numero di Archi (Relazioni uniche): 90

--- Centralità di Grado (Top 3 Settori) ---
Settore: Agricoltura, Centralità: 2.0000
Settore: Consulenza, Centralità: 2.0000
Settore: Distribuzione, Centralità: 2.0000


In [ ]:
# --- 3. Community Detection (Algoritmo di Louvain) ---

# Nota: Louvain lavora meglio sui grafi non diretti, quindi creiamo una versione non direzionata (Graph)
# Manteniamo i pesi (Numero_Connessioni)
G_undirected = G.to_undirected()

# Trova la partizione (le comunità) utilizzando il peso degli archi
# Utilizziamo la funzione `best_partition` della libreria python-louvain
# La variabile `partition` è un dizionario: {nodo: ID_comunità}
partition = community_louvain.best_partition(G_undirected, weight='weight')
modularity = community_louvain.modularity(partition, G_undirected, weight='weight')

# Raggruppamento dei settori per comunità (filiere)
filiere = {}
for nodo, id_comunita in partition.items():
    if id_comunita not in filiere:
        filiere[id_comunita] = []
    filiere[id_comunita].append(nodo)

print("\n--- Risultato della Community Detection (Filiere Emerse) ---")
print(f"Modularity del network (qualità della divisione): {modularity:.4f}")
print(f"Numero di Filiere (Comunità) identificate: {len(filiere)}")

# Stampa le filiere identificate
for id_comunita, settori_comunita in filiere.items():
    print(f"\nFiliere {id_comunita}:")
    print(", ".join(settori_comunita))

# Aggiunta dell'ID di Comunità come attributo a ogni nodo per la visualizzazione futura
for nodo, id_comunita in partition.items():
    G.nodes[nodo]['filiere_id'] = id_comunita


--- Risultato della Community Detection (Filiere Emerse) ---
Modularity del network (qualità della divisione): 0.0000
Numero di Filiere (Comunità) identificate: 1

Filiere 0:
Agricoltura, Consulenza, Distribuzione, Energia, Finanza, Manifattura, Materie Prime, Retail, Servizi IT, Trasporti & Logistica
